<a href="https://colab.research.google.com/github/ZlatanSU87/Graduate-work.-Forecast-for-the-cafe-chain/blob/main/cafe_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируем библиотеки

In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, date, time

# Путь к файлу на Google Drive
INPUT_CSV = '/content/drive/MyDrive/Cafe_prediction/cafe_holiday.csv'
OUTPUT_CSV = '/content/drive/MyDrive/Cafe_prediction/cafe_with_weather.csv'

# Координаты центра Петрозаводска
LAT, LON = 61.7833, 34.3500
TIMEZONE = 'Europe/Moscow'

df = pd.read_csv(INPUT_CSV)

Приведем имена предприятий к единообразию, а также время

In [ ]:
# Создаем функцию для очистки имен
def clean_shop_name(name):
    parts = name.split()
    if len(parts) > 1 and parts[0].lower() == 'кахви':
        return parts[1]
    elif len(parts) == 1:
        return parts[0]
    else:
        return None

# Применяем очистку к столбцу
df['Торговое предприятие'] = df['Торговое предприятие'].apply(clean_shop_name)

# Посчитаем итоговые значения
print(df['Торговое предприятие'].value_counts())

Торговое предприятие
Энгельса     1525256
Свердлова     463940
Плаза         366958
Тетрис        143875
Анохина        14131
Name: count, dtype: int64


In [ ]:
# используем колонку "Время открытия" и приводим временную колонку к datetime
df['time'] = pd.to_datetime(df['Время открытия'])
# сделать timezone-aware (локальное московское время)
try:
    # если dt уже имеет tz, этот вызов вызовет ошибку, поэтому в try
    df['time'] = df['time'].dt.tz_localize(TIMEZONE)
except Exception:
    # если уже tz-aware или pandas автоматом распознал tz, просто привести к нужному
    df['time'] = df['time'].dt.tz_convert(TIMEZONE)

start_date = df['time'].dt.date.min().isoformat()
end_date   = df['time'].dt.date.max().isoformat()
print('Dates:', start_date, '->', end_date)

Dates: 2021-04-06 -> 2025-09-18


Соберем необходимые данные с помощью Open-Meteo archive API

In [ ]:
# Запрос к Open-Meteo archive API (hourly + daily)
base_url = 'https://archive-api.open-meteo.com/v1/archive'
params = {
    'latitude': LAT,
    'longitude': LON,
    'start_date': start_date,
    'end_date': end_date,
    'timezone': TIMEZONE,
    # hourly variables (чтобы посчитать средние/максимумы/ручную агрегацию)
    'hourly': ','.join([
        'temperature_2m',
        'relativehumidity_2m',
        'precipitation',   # суммарные осадки (включая снег)
        'snowfall',        # снег отдельно (в мм)
        'weathercode',
        'windspeed_10m',
        'windgusts_10m',
        'is_day'           # 0/1 индикатор дневного периода
    ]),
    # daily: sunrise/sunset + агрегаты
    'daily': ','.join([
        'sunrise',
        'sunset',
        'rain_sum',
        'snowfall_sum',
        'precipitation_sum',
        'temperature_2m_max',
        'temperature_2m_min',
        'windspeed_10m_max'
    ])
}

print('Requesting weather from Open-Meteo...')
r = requests.get(base_url, params=params, timeout=60)
r.raise_for_status()
data = r.json()

Requesting weather from Open-Meteo...


In [ ]:
# Собираем hourly в DataFrame
hourly = data.get('hourly', {})
if not hourly or 'time' not in hourly:
    raise RuntimeError('No hourly data returned by API. Response keys: %s' % list(data.keys()))

hourly_df = pd.DataFrame(hourly)
# приводим время к datetime с timezone
hourly_df['time'] = pd.to_datetime(hourly_df['time'])
# API вернул времена в локальной timezone (без tzinfo) — добавим tz
try:
    hourly_df['time'] = hourly_df['time'].dt.tz_localize(TIMEZONE)
except Exception:
    # если уже tz-aware, пропускаем
    pass

In [ ]:
# вычисляем дневные агрегаты из hourly
hourly_df['date'] = hourly_df['time'].dt.date

agg_funcs = {
    'temperature_2m': ['mean', 'max', 'min'],
    'precipitation': 'sum',
    'snowfall': 'sum',
    'windspeed_10m': 'mean',
    'windgusts_10m': 'max',
    'relativehumidity_2m': 'mean'
}
daily_agg = hourly_df.groupby('date').agg(agg_funcs)
# упростим имена колонок
daily_agg.columns = [
    '_'.join(filter(None, col)).strip() for col in daily_agg.columns.values
]
daily_agg = daily_agg.reset_index()
# переименование

daily_agg = daily_agg.rename(columns={
    'temperature_2m_mean': 't_mean',
    'temperature_2m_max': 't_max',
    'temperature_2m_min': 't_min',
    'precipitation_sum' if 'precipitation_sum' in daily_agg.columns else 'precipitation_sum': 'precip',
    'precipitation_sum': 'precip',
    'precipitation_sum': 'precip',
    'precipitation_sum': 'precip',
    'precipitation_sum': 'precip'
})

# Если в daily_agg есть 'precipitation_sum' оно уже переименовано; но мы рассчитали precip из hourly:
if 'precip' not in daily_agg.columns:
    # precipitation sum из hourly
    daily_agg['precip'] = daily_agg['precipitation_sum'] if 'precipitation_sum' in daily_agg.columns else daily_agg.get('precipitation_sum', daily_agg.get('precipitation', np.nan))
# Добавим rain (precip - snowfall) и snow
daily_agg['snow'] = daily_agg.get('snowfall_sum', np.nan)  # если есть daily snowfall_sum
# если нет, то используем сумму hourly snowfall:
if daily_agg['snow'].isna().all():
    possible = [c for c in daily_agg.columns if 'snowfall' in c]
    if possible:
        daily_agg['snow'] = daily_agg[possible[0]]
    else:
        daily_agg['snow'] = 0.0

# compute rain as precip - snow (clamp >=0)
daily_agg['rain'] = (daily_agg['precip'] - daily_agg['snow']).clip(lower=0.0)

# wind_mean / wind_max / humidity
# Our earlier agg produced names 'windspeed_10m_mean' and 'windgusts_10m_max' and 'relativehumidity_2m_mean'
# Map them to user-friendly names:
if 'windspeed_10m_mean' in daily_agg.columns:
    daily_agg = daily_agg.rename(columns={'windspeed_10m_mean': 'wind_mean'})
elif 'windspeed_10m_mean' not in daily_agg.columns and 'windspeed_10m_mean' in hourly_df.columns:
    daily_agg['wind_mean'] = hourly_df.groupby('date')['windspeed_10m'].mean().values

if 'windgusts_10m_max' in daily_agg.columns:
    daily_agg = daily_agg.rename(columns={'windgusts_10m_max': 'wind_max'})
elif 'windgusts_10m_max' not in daily_agg.columns and 'windgusts_10m' in hourly_df.columns:
    daily_agg['wind_max'] = hourly_df.groupby('date')['windgusts_10m'].max().values

if 'relativehumidity_2m_mean' in daily_agg.columns:
    daily_agg = daily_agg.rename(columns={'relativehumidity_2m_mean': 'humidity'})
else:
    # fallback compute from hourly
    daily_agg['humidity'] = hourly_df.groupby('date')['relativehumidity_2m'].mean().values

# weathercode: возьмём наиболее частое значение в течение дня (mode)
def mode_or_nan(s):
    m = s.mode()
    return m.iloc[0] if not m.empty else np.nan

weathercode_daily = hourly_df.groupby('date')['weathercode'].agg(mode_or_nan).reset_index().rename(columns={'weathercode':'weathercode'})
daily_agg = daily_agg.merge(weathercode_daily, on='date', how='left')

In [ ]:
# Добавим sunrise/sunset из daily (чтобы сделать is_daytime)
daily_meta = {}
daily = data.get('daily', {})
if daily:
    # daily keys like 'time', 'sunrise', 'sunset'
    daily_meta_df = pd.DataFrame(daily)
    # ensure date column present and parsed
    daily_meta_df['date'] = pd.to_datetime(daily_meta_df['time']).dt.date
    # parse sunrise/sunset into timezone-aware timestamps
    if 'sunrise' in daily_meta_df.columns and 'sunset' in daily_meta_df.columns:
        # pd.to_datetime will parse strings like '2021-04-06T03:20'
        daily_meta_df['sunrise_ts'] = pd.to_datetime(daily_meta_df['sunrise']).dt.tz_localize(TIMEZONE)
        daily_meta_df['sunset_ts']  = pd.to_datetime(daily_meta_df['sunset']).dt.tz_localize(TIMEZONE)
        daily_meta_df = daily_meta_df[['date','sunrise_ts','sunset_ts']]

        daily_agg = daily_agg.merge(daily_meta_df, on='date', how='left')

In [ ]:
# Merge погодные признаки в исходный датасет по дате
df['date'] = df['time'].dt.date
# ensure daily_agg.date is date type
daily_agg['date'] = pd.to_datetime(daily_agg['date']).dt.date

merged = df.merge(daily_agg, on='date', how='left', suffixes=('','_weather'))

In [ ]:
# вычисляем is_daytime для каждой записи (опираясь на sunrise_ts/sunset_ts)
if 'sunrise_ts' in merged.columns and 'sunset_ts' in merged.columns:
    # сравнения работают с timezone-aware timestamps
    merged['is_daytime'] = merged.apply(
        lambda row: (row['time'] >= row['sunrise_ts']) and (row['time'] <= row['sunset_ts'])
        if pd.notnull(row['sunrise_ts']) and pd.notnull(row['sunset_ts']) else np.nan,
        axis=1
    )
else:
    # fallback: считать дневным по локальному времени (8-22) — кафе работают в 8:00-22:00
    merged['local_hour'] = merged['time'].dt.hour
    merged['is_daytime'] = ((merged['local_hour'] >= 8) & (merged['local_hour'] <= 22)).astype(int)
    merged.drop(columns=['local_hour'], inplace=True)

In [ ]:
# Оставим только нужные колонки погоды и переименуем их в требуемые имена
# Проверим и нормализуем названия, которые могли оказаться в daily_agg
for col in ['t_mean','t_max','t_min','precip','rain','snow','wind_mean','wind_max','humidity','weathercode','is_daytime']:
    if col not in merged.columns:
        merged[col] = np.nan

Записываем результат

In [ ]:
merged.to_csv(OUTPUT_CSV, index=False)
print('Saved merged file to', OUTPUT_CSV)

Saved merged file to /content/drive/MyDrive/Cafe_prediction/cafe_with_weather.csv


In [ ]:
df_weather = pd.read_csv('/content/drive/MyDrive/Cafe_prediction/cafe_with_weather.csv')

Датасет с погодными явлениями

In [ ]:
df_weather.iloc[:, 14:].head(4)

,time,date,t_mean,t_max,t_min,precip,snowfall_sum,wind_mean,wind_max,humidity,snow,rain,weathercode,sunrise_ts,sunset_ts,is_daytime
0,2021-04-06 15:48:05+03:00,2021-04-06,1.9375,3.8,-0.6,2.5,0.0,17.8125,49.0,76.583333,0.0,2.5,51,2021-04-06 05:47:00+03:00,2021-04-06 19:42:00+03:00,True
1,2021-04-06 15:48:05+03:00,2021-04-06,1.9375,3.8,-0.6,2.5,0.0,17.8125,49.0,76.583333,0.0,2.5,51,2021-04-06 05:47:00+03:00,2021-04-06 19:42:00+03:00,True
2,2021-04-06 16:04:03+03:00,2021-04-06,1.9375,3.8,-0.6,2.5,0.0,17.8125,49.0,76.583333,0.0,2.5,51,2021-04-06 05:47:00+03:00,2021-04-06 19:42:00+03:00,True
3,2021-04-06 16:06:59+03:00,2021-04-06,1.9375,3.8,-0.6,2.5,0.0,17.8125,49.0,76.583333,0.0,2.5,51,2021-04-06 05:47:00+03:00,2021-04-06 19:42:00+03:00,True


In [ ]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2514160 entries, 0 to 2514159
Data columns (total 30 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Время открытия        object 
 1   Блюдо                 object 
 2   Количество блюд       float64
 3   Группа блюда          object 
 4   Тип оплаты            object 
 5   Торговое предприятие  object 
 6   Сумма со скидкой, р.  float64
 7   Сумма без скидки, р.  float64
 8   Наценка(%)            float64
 9   Себестоимость, р.     float64
 10  Себестоимость(%)      float64
 11  Дата                  object 
 12  holiday_name          object 
 13  is_holiday            int64  
 14  time                  object 
 15  date                  object 
 16  t_mean                float64
 17  t_max                 float64
 18  t_min                 float64
 19  precip                float64
 20  snowfall_sum          float64
 21  wind_mean             float64
 22  wind_max              float64
 23  humidit